<a href="https://colab.research.google.com/github/mohsenh17/jaxLearning/blob/main/flax/flaxCheckpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade flax orbax jax

In [65]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np

from flax import nnx
import jax.numpy as jnp

from sklearn.preprocessing import MinMaxScaler

import orbax.checkpoint as ocp

import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import io



In [74]:

# Generate a dataset with 100 samples, 10 features, 5 informative, 5 redundant, and 2 classes
X, y = make_classification(n_samples=256, n_features=10, n_informative=8,
                          n_classes=2, random_state=42)
df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
df['Class'] = y
csv_data = df.to_csv(index=True)


In [71]:
class CustomImageDataset(Dataset):
    def __init__(self, dataset, transform=None, target_transform=None):
        completeDF = pd.read_csv(dataset, index_col=0)
        self.labels = pd.DataFrame(completeDF['Class'])
        features_raw = pd.DataFrame(completeDF.drop('Class', axis=1))
        #self.features = pd.DataFrame(completeDF.drop('Class', axis=1))
        scaler = MinMaxScaler()
        self.features = pd.DataFrame(scaler.fit_transform(features_raw))
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = np.array(self.features.iloc[idx, :])
        label = self.labels.iloc[idx, 0]
        if self.transform:
            features = self.transform(features)
        if self.target_transform:
            label = self.target_transform(label)
        return features, label

In [75]:
dataset = CustomImageDataset(dataset=io.StringIO(csv_data))
#dataset = CustomImageDataset(dataset="dataset/riceClassification.csv")
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [.7, 0.1,0.2])
data_loader = DataLoader(train_set, batch_size=4, shuffle=True)

for features, labels in data_loader:
    print("Batch of features has shape: ",features.shape)
    print("Batch of labels has shape: ", labels.shape)
    print(features)
    print(labels)
    break

Batch of features has shape:  torch.Size([4, 10])
Batch of labels has shape:  torch.Size([4])
tensor([[0.6415, 0.3986, 0.3478, 0.3950, 0.2933, 0.2386, 0.3254, 0.7682, 0.4544,
         0.6269],
        [0.5566, 0.5970, 0.3363, 0.5311, 0.5245, 0.3408, 0.7131, 0.6877, 0.4067,
         0.4531],
        [0.4134, 0.6072, 0.2426, 0.4445, 0.2696, 0.3440, 0.2871, 0.4632, 0.5598,
         0.5877],
        [0.6251, 0.7010, 0.6922, 0.5333, 0.7998, 0.2144, 0.6096, 0.5264, 0.4557,
         0.4470]], dtype=torch.float64)
tensor([1, 0, 1, 0])


In [76]:
class MLP(nnx.Module):
    def __init__(self, din, dm1, dm2, dm3, num_classes: int, rngs: nnx.Rngs):
      self.linear1 = nnx.Linear(10, 16, rngs=rngs)
      self.dp1 = nnx.Dropout(rate=0.4, rngs=rngs)
      self.bn1 = nnx.BatchNorm(16, rngs=rngs)
      """self.linear2 = nnx.Linear(16, 32, rngs=rngs)
      self.bn2 = nnx.BatchNorm(32, rngs=rngs)
      self.linear3 = nnx.Linear(32, 16, rngs=rngs)
      self.bn3 = nnx.BatchNorm(16, rngs=rngs)
      self.linear4 = nnx.Linear(16, 1, rngs=rngs)"""


    def __call__(self, x):
      x = self.linear1(x)
      x = nnx.gelu(x)
      x = self.dp1(x)
      x = self.bn1(x)
      """#x = self.dropout(x)
      x = self.linear2(x)
      x = nnx.gelu(x)
      x = self.bn2(x)
      #x = self.dropout(x)
      x = self.linear3(x)
      x = nnx.gelu(x)
      x = self.bn3(x)"""
      return nnx.sigmoid(x)

# Instantiate and test the model
model = MLP(10, 16, 32, 16, 1, rngs=nnx.Rngs(0))
#y = model(x=jnp.ones((3, 10)))

#nnx.display(model)

#model = MLP([2, 4], 1, rngs=nnx.Rngs(0))
y = model(x=jnp.ones((3, 10)))

nnx.display(y)



[[0.5        0.8028934  0.5        0.8043662  0.33058918 0.33053228
  0.33028775 0.33034152 0.5        0.19563586 0.8044264  0.33030894
  0.6697085  0.3302991  0.6696849  0.3302848 ]
 [0.5        0.33131608 0.5        0.3302831  0.33058918 0.80401146
  0.80435956 0.33034152 0.5        0.66971546 0.33024082 0.8043294
  0.6697085  0.80434346 0.19567941 0.8043637 ]
 [0.5        0.33131608 0.5        0.3302831  0.8039304  0.33053228
  0.33028775 0.80428296 0.5        0.66971546 0.33024082 0.33030894
  0.19564575 0.3302991  0.6696849  0.3302848 ]]


In [88]:
import optax
learning_rate = 0.001
momentum = 0.9

model = MLP(10, 16, 32, 16, 1, rngs=nnx.Rngs(0))
optimizer = nnx.Optimizer(model, optax.adam(learning_rate))
metrics = nnx.MultiMetric(
  loss=nnx.metrics.Average('loss'),
)

In [79]:

def loss_fn(model: MLP, batch):
  logits = model(batch['features'])
  loss = optax.sigmoid_binary_cross_entropy(
    logits=logits, labels=batch['labels'].reshape(-1, 1)
  ).mean()
  #loss = (logits - batch['labels'])**2
  #print(logits.pval)
  return loss, logits

@nnx.jit
def train_step(model: MLP, optimizer: nnx.Optimizer, metrics: nnx.MultiMetric, batch):
  """Train for a single step."""
  grad_fn = nnx.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(model, batch)

  metrics.update(loss=loss, logits=logits, labels=batch['labels'])  # In-place updates.
  optimizer.update(grads)  # In-place updates.
  return loss, logits

@nnx.jit
def eval_step(model: MLP, metrics: nnx.MultiMetric, batch):
  loss, logits = loss_fn(model, batch)
  metrics.update(loss=loss, logits=logits, labels=batch['labels'])  # In-place updates.
  return loss

In [80]:
def custom_collate_fn(batch):
    transposed_data = list(zip(*batch))

    labels =  np.array(transposed_data[1])
    features = np.array(transposed_data[0])

    return {"features":features, "labels":labels}

train_set, val_set, test_set = torch.utils.data.random_split(dataset, [.7, 0.1,0.2])

train_ds = DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True, collate_fn=custom_collate_fn)
val_set = DataLoader(val_set, batch_size=4, shuffle=True, drop_last=True, collate_fn=custom_collate_fn)
test_ds = DataLoader(test_set, batch_size=4, shuffle=True, drop_last=True, collate_fn=custom_collate_fn)

# test
batch_data = next(iter(train_ds))
imgs = batch_data['features']
lbls = batch_data['labels']
print(imgs.shape, imgs[0].dtype, lbls.shape, lbls[0].dtype)
print(lbls)

#loss = train_step(model, optimizer, metrics, batch_data)
loss, logits = loss_fn(model, batch_data)
print(loss.shape, logits.shape)
print(f'{loss = }')
#print(f'{logits = }')
print(f'{optimizer.step.value = }')


(64, 10) float64 (64,) int64
[0 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1
 1 1 0 0 0 1 1 0 0 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0 0 0]
() (64, 16)
loss = Array(0.72493005, dtype=float32)
optimizer.step.value = Array(0, dtype=uint32)


In [81]:
import jax
import orbax.checkpoint as ocp
ckpt_dir = ocp.test_utils.erase_and_create_empty('/content/my-checkpoints/')
metrics_history = {
  'train_loss': [],
  'train_accuracy': [],
  'train_precision': [],
  'train_recall': [],
  'train_f1': [],
  'val_loss': [],
  'val_accuracy': [],
  'val_precision': [],
  'val_recall': [],
  'val_f1': [],
  'test_loss': [],
  'test_accuracy': [],
  'test_precision': [],
  'test_recall': [],
  'test_f1': [],
}

num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_ds:
      loss, logits = train_step(model, optimizer, metrics, batch)

    for metric, value in metrics.compute().items():
      metrics_history[f'train_{metric}'].append(value)

    metrics.reset()
    print(
      f"[train] epoch: {epoch}, "
      f"loss: {metrics_history['train_loss'][-1]}, "
    )

[train] epoch: 0, loss: 0.7277255058288574, 
[train] epoch: 1, loss: 0.7444443106651306, 
[train] epoch: 2, loss: 0.7387948036193848, 
[train] epoch: 3, loss: 0.7119072675704956, 
[train] epoch: 4, loss: 0.7009329199790955, 
[train] epoch: 5, loss: 0.7275881767272949, 
[train] epoch: 6, loss: 0.7228589057922363, 
[train] epoch: 7, loss: 0.7303940057754517, 
[train] epoch: 8, loss: 0.7135761976242065, 
[train] epoch: 9, loss: 0.7185161113739014, 


In [82]:
model.eval() # Switch to evaluation mode.

@nnx.jit
def pred_step(model: MLP, batch):
  logits = model(batch['features'])
  return logits

test_ds = DataLoader(test_set, batch_size=32, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)

ypred = []
label = []
for test_batch in test_ds:
  logits = pred_step(model, test_batch)
  #print(np.ravel(logits))
  #print(np.ravel(logits))
  #break
  ypred.extend(np.ravel(logits))
  label.extend(np.ravel(test_batch["labels"]))
  #print(logits, test_batch["labels"])
  #break
binary_ypred = np.where(np.array(ypred) > 0.5, 1, 0)
print(len(label), sum(label))
accuracy = sum([1 for pred, true in zip(binary_ypred, label) if pred == true]) / len(label)
print(f"Accuracy: {accuracy:.4f}")

32 17
Accuracy: 0.4688


In [83]:
import jax
from orbax.checkpoint.type_handlers import TypeHandler
from orbax.checkpoint.type_handlers import register_type_handler

state = nnx.state(model)
_, state = nnx.split(model)
nnx.display(state['dp1'])
prng_key_value = state["dp1"]["rngs"]["default"]["key"].value
#print(prng_key_value.dtype)
state["dp1"]["rngs"]["default"]["key"] = nnx.VariableState(
    type=nnx.Param, value=jax.random.key_data(prng_key_value), tag='default'
)
nnx.display(state)
# Save the parameters
ckpt_dir = ocp.test_utils.erase_and_create_empty('/content/my-checkpoints/')
checkpointer = ocp.PyTreeCheckpointer()
checkpointer.save('/content/my-checkpoints/state', state)

State({
  'rngs': {
    'default': {
      'count': VariableState(
        type=RngCount,
        value=Array(25, dtype=uint32),
        tag='default'
      ),
      'key': VariableState(
        type=RngKey,
        value=Array((), dtype=key<fry>) overlaying:
        [0 0],
        tag='default'
      )
    }
  }
})
State({
  'bn1': {
    'bias': VariableState(
      type=Param,
      value=Array([-0.01851453, -0.01809671, -0.01805194, -0.01850327, -0.01832869,
             -0.01856782, -0.01792235, -0.01848269, -0.01820147, -0.01813109,
             -0.01774389, -0.01871058, -0.01857002, -0.01832548, -0.01838941,
             -0.01777863], dtype=float32)
    ),
    'mean': VariableState(
      type=BatchStat,
      value=Array([-2.9075747e-02, -3.0570619e-02, -7.8527381e-05, -1.8928114e-02,
             -3.1981084e-02,  7.6465530e-04,  1.1133681e-02, -9.8525900e-03,
              2.2586300e-03, -1.9554241e-02,  3.5659537e-02, -1.5808607e-02,
             -1.6209841e-02, -2.9383343e-0

In [91]:
abstract_model = nnx.eval_shape(lambda: model)
graphdef, abstract_state = nnx.split(abstract_model)
print('The abstract NNX state (all leaves are abstract arrays):')
nnx.display(abstract_state)

state_restored = checkpointer.restore(ckpt_dir / 'state', abstract_state)
nnx.display(state_restored['dp1'])
prng_key_value = state["dp1"]["rngs"]["default"]["key"].value
state_restored["dp1"]["rngs"]["default"]["key"] = nnx.VariableState(
    type=nnx.Param, value=jax.random.wrap_key_data(prng_key_value), tag='default'
)

#jax.tree.map(np.testing.assert_array_equal, state, state_restored)
print('NNX State restored: ')
nnx.display(state_restored)

# The model is now good to use!
model = nnx.merge(graphdef, state_restored)

The abstract NNX state (all leaves are abstract arrays):
State({
  'bn1': {
    'bias': VariableState(
      type=Param,
      value=ShapeDtypeStruct(shape=(16,), dtype=float32)
    ),
    'mean': VariableState(
      type=BatchStat,
      value=ShapeDtypeStruct(shape=(16,), dtype=float32)
    ),
    'scale': VariableState(
      type=Param,
      value=ShapeDtypeStruct(shape=(16,), dtype=float32)
    ),
    'var': VariableState(
      type=BatchStat,
      value=ShapeDtypeStruct(shape=(16,), dtype=float32)
    )
  },
  'dp1': {
    'rngs': {
      'default': {
        'count': VariableState(
          type=RngCount,
          value=ShapeDtypeStruct(shape=(), dtype=uint32),
          tag='default'
        ),
        'key': VariableState(
          type=Param,
          value=ShapeDtypeStruct(shape=(), dtype=key<fry>),
          tag='default'
        )
      }
    }
  },
  'linear1': {
    'bias': VariableState(
      type=Param,
      value=ShapeDtypeStruct(shape=(16,), dtype=float32)


In [90]:
import jax

metrics_history = {
  'train_loss': [],
  'train_accuracy': [],
  'train_precision': [],
  'train_recall': [],
  'train_f1': [],
  'val_loss': [],
  'val_accuracy': [],
  'val_precision': [],
  'val_recall': [],
  'val_f1': [],
  'test_loss': [],
  'test_accuracy': [],
  'test_precision': [],
  'test_recall': [],
  'test_f1': [],
}

num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_ds:
      loss, logits = train_step(model, optimizer, metrics, batch)

    for metric, value in metrics.compute().items():
      metrics_history[f'train_{metric}'].append(value)

    metrics.reset()
    print(
      f"[train] epoch: {epoch}, "
      f"loss: {metrics_history['train_loss'][-1]}, "
    )

TypeError: grad requires real- or complex-valued inputs (input dtype that is a sub-dtype of np.inexact), but got key<fry>. If you want to use Boolean- or integer-valued inputs, use vjp or set allow_int to True.